In [ ]:
import pandas as pd
import numpy as np

: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Load the dataset
df=pd.read_csv("customer_segmentation.csv")

In [ ]:
#Display the first few rows of the dataset
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isna().sum().sum()

In [ ]:
df.describe()

In [ ]:
df["Education"].value_counts()

In [ ]:
df["Marital_Status"].value_counts()

In [ ]:
df["Dt_Customer"]=pd.to_datetime(df["Dt_Customer"],dayfirst=True)

In [ ]:
df.info()

In [ ]:
df["Age"]=2025-df["Year_Birth"]

In [ ]:
df["Age"]

In [ ]:
df["Total_Children"]=df["Kidhome"]+df["Teenhome"]

In [ ]:
df["Total_Children"]

In [ ]:
spend_cols=['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']

In [ ]:
df["Total_Spending"]=df[spend_cols].sum(axis=1)

In [ ]:
df["Total_Spending"]

In [ ]:
df["Customer_Since"]=(pd.Timestamp("today") - df["Dt_Customer"]).dt.days

In [ ]:
df["Customer_Since"]

In [ ]:
sns.histplot(df["Age"],bins=30,kde=True)
plt.title("Age Distribution")
plt.show()

In [ ]:
sns.histplot(df["Income"],bins=30,kde=True)
plt.title("Income Distribution")
plt.show()

In [ ]:
sns.histplot(df["Total_Spending"],bins=30,kde=True)
plt.title("Total Spending Distribution")
plt.show()

In [ ]:
sns.boxplot(x="Education",y="Income",data=df)
plt.xticks(rotation=45)
plt.title("Income by Education Level")
plt.show()

In [ ]:
sns.boxplot(x="Marital_Status",y="Total_Spending",data=df)
plt.xticks(rotation=45)
plt.title("Total Spending by Marital Status")
plt.show()

In [ ]:
df.columns

In [ ]:
corr=df[["Income","Age","Recency","Total_Spending","NumWebPurchases","NumStorePurchases"]].corr()

In [ ]:
corr

In [ ]:
sns.heatmap(corr,annot=True,cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
pivot_income= df.pivot_table(index="Education",values="Income",columns="Marital_Status",aggfunc="mean")

In [ ]:
pivot_income

In [ ]:
sns.heatmap(pivot_income,annot=True,fmt=".0f",cmap="YlGnBu")
plt.title("Average Income by Education and Marital Status")
plt.show()

In [ ]:
group1=df.groupby("Education")["Total_Spending"].mean().sort_values(ascending=False)

In [ ]:
group1

In [ ]:
group1.plot(kind="bar",color="skyblue")
plt.title("Average Total Spending by Education Level")
plt.ylabel("Average Total Spending")
plt.xlabel("Education Level")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df["AcceptedAny"]=df[["AcceptedCmp1","AcceptedCmp2","AcceptedCmp3","AcceptedCmp4","AcceptedCmp5","Response"]].sum(axis=1)

In [ ]:
df["AcceptedAny"].unique()

In [ ]:
df["AcceptedAny"]=df["AcceptedAny"].apply(lambda x: 1 if x>0 else 0)

In [ ]:
df["AcceptedAny"].unique()

In [ ]:
group2=df.groupby("Marital_Status")["AcceptedAny"].mean().sort_values(ascending=False)

In [ ]:
group2

In [ ]:
group2.plot(kind="bar",color="lightgreen")
plt.title("Proportion of Customers Accepting Any Campaign by Marital Status")
plt.ylabel("Proportion Accepting Any Campaign")
plt.xlabel("Marital Status")
plt.xticks(rotation=45)
plt.show()

In [ ]:
bins=[10,30,40,50,60,70,90]

In [ ]:
labels=["18-29","30-39","40-49","50-59","60-69","70+"]

In [ ]:
df["AgeGroup"]=pd.cut(df["Age"],bins=bins,labels=labels)

In [ ]:
df["AgeGroup"]

In [ ]:
group3=df.groupby("AgeGroup")["Income"].mean()

In [ ]:
group3

In [ ]:
group3.plot(kind="barh",color="coral")
plt.title("Average Income by Age Group")
plt.ylabel("Average Income")
plt.xlabel("Age Group")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
features=["Age","Income","Total_Spending","NumWebPurchases","NumStorePurchases","NumWebVisitsMonth","Recency"]

In [ ]:
X=df[features].copy()

In [ ]:
X

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()


In [ ]:
X_scaled=scaler.fit_transform(X)

In [ ]:
X_scaled

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
wcss=[]

In [ ]:
for i in range(2,10):
    kmeans=KMeans(n_clusters=i)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

In [ ]:
wcss

In [ ]:
plt.plot(range(2,10),wcss,marker='o')
plt.title("Elbow Method for Optimal k")
plt.xlabel("Number of clusters (k)")
plt.ylabel("WCSS")
plt.show()

In [ ]:
kmeans=KMeans(n_clusters=6)
df["Clusters"]=kmeans.fit_predict(X_scaled)


In [ ]:
df.head()

In [ ]:
clusters_summary=df.groupby("Clusters")[features].mean()

In [ ]:
clusters_summary

In [ ]:
df["Clusters"].value_counts()

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
pca_data=pca.fit_transform(X_scaled)
df["PCA1"],df["PCA2"]=pca_data[:,0],pca_data[:,1]

In [ ]:
pca_data

In [ ]:
sns.scatterplot(x="PCA1",y="PCA2",hue="Clusters",data=df,palette="Set1")
plt.title("Customer Segments based on PCA")
plt.show()

In [ ]:
clusters_summary

In [ ]:
import joblib
joblib.dump(kmeans,"kmeans_model.pkl")
joblib.dump(scaler,"scaler.pkl")